## Body in front of Spacecraft

Its important to predicte tiem intervals that the radiotelescope would not be able to se the spacecraft.

In this notebook we predict when Jupiter passes in front of Juno, seeing from Earth.

#### - Imports e kernels

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from tudatpy.interface import spice
from tudatpy import dynamics
from tudatpy import numerical_simulation
from tudatpy.dynamics import environment_setup, propagation_setup
from tudatpy.astro import element_conversion
from tudatpy import constants
from tudatpy.util import result2array
from tudatpy.astro.time_representation import DateTime

spice.load_standard_kernels()

#### - Creating bodies

In [2]:
simulation_start_epoch = DateTime(2016, 10, 5).to_epoch()
propagation_start_epoch = simulation_start_epoch - 3600.0
simulation_end_epoch = simulation_start_epoch + 60*24*3600 #60 days (covering a Juno's orbit)
propagation_end_epoch = simulation_end_epoch + 3600.0

bodies_to_create = ["Sun","Jupiter","Earth"]

global_frame_origin = "Jupiter"
global_frame_orientation = "ECLIPJ2000"

body_settings = environment_setup.get_default_body_settings(
    bodies_to_create,
    global_frame_origin,
    global_frame_orientation
)

body_settings.add_empty_settings("Juno")

body_settings.get("Juno").constant_mass = 2e3  #2000 kg

jupiter_gravitational_parameter = constants.GRAVITATIONAL_CONSTANT * 1.898e27  

semi_major_axis = 1.07e9
eccentricity = 0.95
inclination = np.deg2rad(90.0)
argument_of_periapsis = np.deg2rad(90.0)
longitude_of_ascending_node = np.deg2rad(180.0)
true_anomaly = np.deg2rad(0.0)

keplerian_elements = [
    semi_major_axis,
    eccentricity,
    inclination,
    argument_of_periapsis,
    longitude_of_ascending_node,
    true_anomaly
]

juno_ephemeris_settings = environment_setup.ephemeris.keplerian(
    keplerian_elements,
    propagation_start_epoch,
    jupiter_gravitational_parameter,
    global_frame_origin,
    global_frame_orientation
)

reference_area_radiation = 50.0  # m²
radiation_pressure_coefficient = 1.2
occulting_bodies_dict = {"Sun": ["Jupiter", "Earth"]}

body_settings.get("Juno").radiation_pressure_target_settings = (
    environment_setup.radiation_pressure.cannonball_radiation_target(
        reference_area_radiation,
        radiation_pressure_coefficient,
        occulting_bodies_dict
    )
)

body_settings.get("Juno").ephemeris_settings = juno_ephemeris_settings

bodies = environment_setup.create_system_of_bodies(body_settings)

In [3]:
dss14_altitude = 1071.0
dss14_latitude = np.deg2rad(35.4269)
dss14_longitude = np.deg2rad(243.1105)

environment_setup.add_ground_station(
    bodies.get_body("Earth"),
    "DSS-14",
    np.array([dss14_altitude, dss14_latitude, dss14_longitude])
)

#### - Occultation conditions

In [5]:
def check_jupiter_occultation(time, bodies):

    juno_state = bodies.get('Juno').state_in_base_frame_from_ephemeris(time)
    jupiter_state = bodies.get('Jupiter').state_in_base_frame_from_ephemeris(time)
    earth_state = bodies.get('Earth').state_in_base_frame_from_ephemeris(time)

    # CORRECT LINE OF SIGHT VECTORS
    r_earth_to_juno = juno_state - earth_state
    r_earth_to_jupiter = jupiter_state - earth_state

    separation_angle = np.arccos(
        np.clip(
            np.dot(r_earth_to_juno, r_earth_to_jupiter) /
            (np.linalg.norm(r_earth_to_juno) * np.linalg.norm(r_earth_to_jupiter)),
            -1.0, 1.0
        )
    )

    jupiter_radius = bodies.get_body("Jupiter").shape_model.average_radius
    jupiter_angular_radius = np.arcsin(
        jupiter_radius / np.linalg.norm(r_earth_to_jupiter)
    )

    return separation_angle < jupiter_angular_radius


#### - Detect occultations

In [6]:
def find_occultation_periods(start_time, end_time, step_size=600):
    times = np.arange(start_time, end_time, step_size)
    occultation_status = []
    
    for time in times:
        is_occulted = check_jupiter_occultation(time, bodies)
        occultation_status.append((time, is_occulted))
    
    occultation_periods = []
    in_occultation = False
    period_start = None
    
    for time, occulted in occultation_status:
        if occulted and not in_occultation:
            period_start = time
            in_occultation = True
        elif not occulted and in_occultation:
            occultation_periods.append((period_start, time))
            in_occultation = False
            
    if in_occultation:
        occultation_periods.append((period_start, times[-1]))
        
    return occultation_periods

#### - Defining Juno trajectory

In [7]:
spacecraft_name = "Juno"

acceleration_settings = {
    spacecraft_name: {
        "Jupiter": [
            propagation_setup.acceleration.point_mass_gravity(),
            propagation_setup.acceleration.spherical_harmonic_gravity(2, 0)
        ],
        "Sun": [
            propagation_setup.acceleration.point_mass_gravity()
        ]
    }
}

acceleration_models = propagation_setup.create_acceleration_models(
    bodies,
    acceleration_settings,
    bodies_to_propagate=[spacecraft_name],
    central_bodies=["Jupiter"]
)

In [8]:
jupiter_gravitational_parameter = bodies.get_body("Jupiter").gravitational_parameter

initial_keplerian_elements = np.array([
    20.0e7,                 # semi-major axis [m]
    0.6,                    # eccentricity
    np.deg2rad(90.0),       # inclination
    np.deg2rad(0.0),        # argument of periapsis
    np.deg2rad(0.0),        # longitude of ascending node
    np.deg2rad(0.0)         # true anomaly
])

initial_cartesian_state = element_conversion.keplerian_to_cartesian(
    initial_keplerian_elements,
    jupiter_gravitational_parameter
)

In [9]:
integrator_settings = propagation_setup.integrator.runge_kutta_4(
    simulation_start_epoch,
    60.0
)

propagator_settings = propagation_setup.propagator.translational(
    central_bodies=["Jupiter"],
    acceleration_models=acceleration_models,
    bodies_to_integrate=[spacecraft_name],
    initial_states=initial_cartesian_state,
    initial_time=simulation_start_epoch,
    integrator_settings=integrator_settings,
    termination_settings=propagation_setup.propagator.time_termination(
        simulation_end_epoch
    )
)

dynamics_simulator = numerical_simulation.create_dynamics_simulator(
    bodies,
    propagator_settings
)

state_history = dynamics_simulator.state_history

state_array = result2array(state_history)
time = state_array[:, 0]
states = state_array[:, 1:]

In [10]:
start_epoch = DateTime(2016, 10, 5).to_epoch()
end_epoch = start_epoch + (60 * 24 * 3600)

periods = find_occultation_periods(start_epoch, end_epoch)

print("\n" + "="*50)
print("OCCULTATIONS LIST (RADIO SCIENCE)")
print("="*50)

if not periods:
    print("No detected occultations.")
else:
    for i, (t_start, t_end) in enumerate(periods):
        utc_start = DateTime.from_epoch(t_start).iso_string()
        utc_end = DateTime.from_epoch(t_end).iso_string()

        duration = (t_end - t_start) / 3600.0
        
        print(f"Event {i+1}:")
        print(f"  Beggning (Immersion): {utc_start}")
        print(f"  End    (Emersion) : {utc_end}")
        print(f"  Total   Duration : {duration:.2f} hours")
        print("-" * 30)

print("="*50)


OCCULTATIONS LIST (RADIO SCIENCE)
Event 1:
  Beggning (Immersion): 2016-10-05 12:00:00.000000000000000
  End    (Emersion) : 2016-10-05 12:10:00.000000000000000
  Total   Duration : 0.17 hours
------------------------------
Event 2:
  Beggning (Immersion): 2016-10-12 13:40:00.000000000000000
  End    (Emersion) : 2016-10-12 15:50:00.000000000000000
  Total   Duration : 2.17 hours
------------------------------
Event 3:
  Beggning (Immersion): 2016-10-19 17:20:00.000000000000000
  End    (Emersion) : 2016-10-19 19:30:00.000000000000000
  Total   Duration : 2.17 hours
------------------------------
Event 4:
  Beggning (Immersion): 2016-10-26 21:00:00.000000000000000
  End    (Emersion) : 2016-10-26 23:10:00.000000000000000
  Total   Duration : 2.17 hours
------------------------------
Event 5:
  Beggning (Immersion): 2016-11-03 00:40:00.000000000000000
  End    (Emersion) : 2016-11-03 02:50:00.000000000000000
  Total   Duration : 2.17 hours
------------------------------
Event 6:
  Begg

### Finding occultations means simulating the ingress and egress of the signal. In radio science, these are the moments when the signal frequency (Doppler) undergoes a violent shift due to the planet's atmosphere. Without occultation, the radio telescope would simply be tracking a probe in empty space.